In [5]:
import torch

from shadowing import (
    RelativeMSE, Foveal, PathShadowing, PredictionContext, 
    realized_variance, ArrayType
)

def _torch(x: ArrayType) -> torch.Tensor:
    """ Convert x to a torch float tensor. """
    if isinstance(x, torch.Tensor):
        return x
    return torch.tensor(x, dtype=torch.float32)

In [7]:
# PathDistance
distance = RelativeMSE()

x = torch.randn(8,34)
y = torch.randn(128,512,34)

ds1, idces1 = distance.forward_topk(x, y, k=32, n_splits=32)
ds2, idces2 = distance.forward_topk(x, y, k=64, n_splits=64)

assert torch.equal(ds1, ds2[:,:ds1.shape[1]])
assert torch.equal(idces1, idces2[:,:idces1.shape[1],:])

In [8]:
# shadowing correctly select paths
x_context = torch.randn(8, 1, 126)
x_dataset = torch.randn(32, 1, 4096)
embedding = Foveal(alpha=1.15, beta=0.9, max_context=126)
distance = RelativeMSE()
obj = PathShadowing(embedding, distance, x_dataset, context=PredictionContext(horizon=252))
distances, paths = obj.shadow(x_context, n_splits=1, k=1024, cuda=False)

# re-compute the distance
paths = _torch(paths)
paths = obj.context.select_in_context(paths)
paths = embedding(paths.view(-1, *paths.shape[2:]))
paths = paths.view(*distances.shape, -1)
x_context = _torch(x_context)
x_context = embedding(x_context)
ds_test = distance(x_context, paths)
assert torch.allclose(ds_test, _torch(distances), rtol=1e-2)

In [9]:
# execution time
x_context = torch.randn(1, 1, 126)
x_dataset = torch.randn(131072, 1, 4096)
embedding = Foveal(alpha=1.15, beta=0.9, max_context=126)
distance = RelativeMSE()
obj = PathShadowing(embedding, distance, x_dataset, context=PredictionContext(horizon=252))
to_predict = lambda x: realized_variance(x, Ts=[2,7,252], vol=False)
pred, pred_std = obj.predict(x_context, k=10000, to_predict=to_predict, eta=0.1,
                             n_context_splits=1, n_dataset_splits=8, cuda=True)

100%|██████████| 1/1 [00:02<00:00,  2.29s/it]
